# Data Cleaning 

#### 1. Import pandas library.

In [2]:
import pandas as pd

#### 2. Import the users table.

In [3]:
users = pd.read_csv("../data/users.csv", index_col=0)

In [8]:
users.head(10)
len(users)

40325

#### 3. Rename Id column to userId.

#### 4. Import the posts table. 

In [5]:
posts = pd.read_csv("../data/posts.csv", index_col=0)

In [9]:
posts.head()
len(posts)

91976

#### 5. Rename Id column to postId and OwnerUserId to userId.

#### 6. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

#### 7. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users.merge(right=posts, how="outer", on="userId")

,userId,Reputation,Views,UpVotes,DownVotes,PostId,Score,ViewCount,CommentCount
0,-1.0,1.0,0.0,5007.0,1920.0,2175.0,0.0,NaN,0.0
1,-1.0,1.0,0.0,5007.0,1920.0,8576.0,0.0,NaN,0.0
2,-1.0,1.0,0.0,5007.0,1920.0,8578.0,0.0,NaN,0.0
3,-1.0,1.0,0.0,5007.0,1920.0,8981.0,0.0,NaN,0.0
4,-1.0,1.0,0.0,5007.0,1920.0,8982.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
110313,NaN,NaN,NaN,NaN,NaN,114678.0,0.0,20.0,2.0
110314,NaN,NaN,NaN,NaN,NaN,114812.0,0.0,16.0,1.0
110315,NaN,NaN,NaN,NaN,NaN,114815.0,1.0,14.0,5.0
110316,NaN,NaN,NaN,NaN,NaN,115225.0,0.0,8.0,0.0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 